In [1]:
# --- Load environment variables ---
import os 
import pandas as pd 

from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not set in .env file")

# Set OpenAI API key for langchain or crewai usage
os.environ["OPENAI_API_KEY"] = openai_api_key

In [2]:
## load the files 

financials_df = pd.read_csv('HealthTrackAI_Financials.csv')

In [3]:
from PyPDF2 import PdfReader

reader = PdfReader("HealthTrackAI_PitchDeck.pdf")

In [4]:
# Extract all text from the PDF
pdf_text = ""
for page in reader.pages:
    pdf_text += page.extract_text() + "\n"

# Extract specific sections from the raw PDF text using keyword-based slicing
lines = pdf_text.splitlines()

# Initialize containers
pitch_summary_lines = []
market_insights_lines = []
risks_lines = []

section = None
for line in lines:
    if "Pitch Summary" in line:
        section = "pitch"
        continue
    elif "Market Insights" in line:
        section = "market"
        continue
    elif "Identified Risks" in line:
        section = "risks"
        continue

    if section == "pitch":
        pitch_summary_lines.append(line)
    elif section == "market":
        market_insights_lines.append(line)
    elif section == "risks" and line.strip().startswith("-"):
        risks_lines.append(line.strip("- ").strip())

# Assign to variables
pitch_summary = " ".join(pitch_summary_lines).strip()
market_insights = " ".join(market_insights_lines).strip()
risks = risks_lines

In [5]:
pitch_summary

'HealthTrack AI is a healthtech startup that uses AI to provide predictive analytics  for patient readmission and hospital resource optimization. Their target market includes  hospitals, clinics, and insurance providers. They operate on a SaaS model.'

In [6]:
market_insights

'The global healthcare analytics market is projected to grow at 15.3% CAGR,  reaching $75B by 2030. Key competitors include MedixAI, PredictCare, and Healthlytics. Regulatory landscape is stable but requires HIPAA and local compliance.'

In [7]:
risks


['High customer acquisition cost',
 'Reliance on a small number of pilot clients',
 'Early-stage founding team with limited domain experience',
 'Data privacy and compliance risks']

In [8]:
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager

In [9]:
## Define Agents 

market_research_agent = AssistantAgent(
    name="Market_Researcher",
    system_message="You are a market analyst. Summarize the industry trends and competitors for HealthTrack AI",
    llm_config={"model": "gpt-4"},
)

finace_agent = AssistantAgent(
    name="Finace_Expert",
    system_message="You are a financial analyst. Evaluate the startup's revenue, burn rate, margins, valuations",
    llm_config={"model": "gpt-4"},
)

risk_agent = AssistantAgent(
    name="Risk_advisor_agent",
    system_message="You are a risk expert. Based on the provided data, summarize top 3 risks and mititgation strategies",
    llm_config={"model": "gpt-4"}
)

memo_draft_agent = AssistantAgent(
    name="MemoWriterAgent",
    system_message="You are an investment stratetist. Based on the provided data, write a final 1 pager memo for the startup combining all data",
    llm_config={"model": "gpt-4"}
)

## user Proxy

userproxy = UserProxyAgent(
    name = 'Arpit',
    human_input_mode="NEVER",
    code_execution_config={"use_docker":False}
)

In [10]:
## Define the group chat 

groupchat = GroupChat(
    agents=[userproxy, market_research_agent, finace_agent, risk_agent, memo_draft_agent],
    messages=[],
    max_round=5
)

In [11]:
manager = GroupChatManager(groupchat=groupchat, llm_config={"model": "gpt-4"})
userproxy.initiate_chat(manager)


Arpit (to chat_manager):



--------------------------------------------------------------------------------

Next speaker: Market_Researcher

Market_Researcher (to chat_manager):

HealthTrack AI operates within the rapidly growing Healthcare Technology sector, particularly in the subcategory of Artificial Intelligence (AI). According to Global Market Insights, the AI in healthcare market size crossed USD 8.5 billion in 202/B and is predicted to grow at a CAGR of over 41.9% from 2021 to 2027. The market is being powered by increasing healthcare costs, growing demand for precision medicine, and a surge in venture capital investments. 

Key industry trends include: 

1. Predictive Healthcare: The ability to predict outcomes based on data analysis is becoming a major focus, allowing healthcare providers to make informed decisions about patient care.
2. Increased Automation: AI is being used to automate various tasks in healthcare from administrative tasks like scheduling appointments to m

ChatResult(chat_id=None, chat_history=[{'content': '', 'role': 'assistant', 'name': 'Arpit'}, {'content': "HealthTrack AI operates within the rapidly growing Healthcare Technology sector, particularly in the subcategory of Artificial Intelligence (AI). According to Global Market Insights, the AI in healthcare market size crossed USD 8.5 billion in 202/B and is predicted to grow at a CAGR of over 41.9% from 2021 to 2027. The market is being powered by increasing healthcare costs, growing demand for precision medicine, and a surge in venture capital investments. \n\nKey industry trends include: \n\n1. Predictive Healthcare: The ability to predict outcomes based on data analysis is becoming a major focus, allowing healthcare providers to make informed decisions about patient care.\n2. Increased Automation: AI is being used to automate various tasks in healthcare from administrative tasks like scheduling appointments to more complex tasks like interpreting medical images or data.\n3. Perso